In [ ]:
import pandas as pd
df= pd.read_csv("/content/drive/MyDrive/412/output_2.csv")

#converting size value from string to float
def convert_size(size):
    if 'M' in size:
        return float(size.replace('M', '')) * 1024 * 1024
    elif 'k' in size:
        return float(size.replace('k', '')) * 1024
    else:
        return size

df['Size'] = df['Size'].apply(convert_size)
df['Size'] = pd.to_numeric(df['Size'], errors='coerce')

#removing + and , from the installs column
df['Installs'] = df['Installs'].str.replace('+', '').str.replace(',', '')

<ipython-input-28-5d74ffa200ed>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Installs'] = df['Installs'].str.replace('+', '').str.replace(',', '')


In [ ]:
#scaling the values in between 0 to 1
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
df['Installs'] = scaler.fit_transform(df[['Installs']])
df['Rating'] = scaler.fit_transform(df[['Rating']])
df['Size'] = scaler.fit_transform(df[['Size']])

In [ ]:
#dividing into 3 class, A is best, C is worst
df['Rating'] = pd.cut(df['Rating'], bins=[0, 1/3, 2/3, 1], labels=['C', 'B', 'A'], include_lowest=True)
df['Installs'] = pd.cut(df['Installs'], bins=[0, 1/3, 2/3, 1], labels=['C', 'B', 'A'], include_lowest=True)
df['Size'] = pd.cut(df['Size'], bins=[0, 1/3, 2/3, 1], labels=['A', 'B', 'C'], include_lowest=True)

In [ ]:
#adding column App_Category in to df. here, the output of this column is based on the majority value of the other columns.
def new_value(row):
    values = list(row.values)
    if values.count('A') > values.count('B') and values.count('A') > values.count('C'):
        return 'A'
    elif values.count('B') > values.count('C'):
        return 'B'
    else:
        return 'C'

df['App_Category'] = df.apply(new_value, axis=1)

In [ ]:
df.head(30)

,App,Rating,Size,Installs,App_Category
0,BM speed test,A,A,C,A
1,EP Church Annapolis,A,A,C,A
2,Morse Player,A,A,C,A
3,Railroad Radio Vancouver BC,A,A,C,A
4,CP Trivia,A,A,C,A
5,ER Assist,A,A,C,A
6,CG Jobs,A,A,C,A
7,CG - Chemistry free,A,A,C,A
8,CG FM,A,A,C,A
9,DF Glue Board,A,A,C,A


In [ ]:
df['App_Category'].value_counts()

A    4614
C    2298
B     117
Name: App_Category, dtype: int64

In [ ]:
df.to_csv('/content/drive/MyDrive/412/new.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Convert categorical variables into numerical variables
le = LabelEncoder()
df['Rating'] = le.fit_transform(df['Rating'])
df['Size'] = le.fit_transform(df['Size'])
df['Installs'] = le.fit_transform(df['Installs'])
df['App_Category'] = le.fit_transform(df['App_Category'])

# Split the data into train and test sets
X = df[['Rating', 'Size', 'Installs']]
y = df['App_Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set and calculate the accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy*100:.2f}%')

Test Accuracy: 100.00%


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)
dt_predictions = dt_classifier.predict(X_test)

# Naive Bayes Classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
nb_predictions = nb_classifier.predict(X_test)

# Evaluate and print the accuracy of both classifiers
dt_accuracy = accuracy_score(y_test, dt_predictions)*100
nb_accuracy = accuracy_score(y_test, nb_predictions)*100

print(f'Decision Tree Classifier Accuracy: {dt_accuracy:.2f}')
print(f'Naive Bayes Classifier Accuracy: {nb_accuracy:.2f}')

# Optionally, you can print additional evaluation metrics
print("\nDecision Tree Classification Report:")
print(classification_report(y_test, dt_predictions))

print("\nNaive Bayes Classification Report:")
print(classification_report(y_test, nb_predictions))


Decision Tree Classifier Accuracy: 100.00
Naive Bayes Classifier Accuracy: 100.00

Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       924
           1       1.00      1.00      1.00        22
           2       1.00      1.00      1.00       460

    accuracy                           1.00      1406
   macro avg       1.00      1.00      1.00      1406
weighted avg       1.00      1.00      1.00      1406


Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       924
           1       1.00      1.00      1.00        22
           2       1.00      1.00      1.00       460

    accuracy                           1.00      1406
   macro avg       1.00      1.00      1.00      1406
weighted avg       1.00      1.00      1.00      1406



**MODEL**

In [28]:
import pandas as pd
df= pd.read_csv("/content/drive/MyDrive/412/category.csv")

In [29]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [30]:
#split dataset

cols = ['Rating','Size','Installs']
X = df[cols]
y = df['Category']

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [31]:
#Naive Bayes

model1 = GaussianNB()
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred)*100)
print("Precision:", precision_score(y_test, y_pred, average='macro')*100)
print("Recall:", recall_score(y_test, y_pred, average='macro')*100)
print("F1 Score:", f1_score(y_test, y_pred, average='macro')*100)

Accuracy: 70.34718269778031
Precision: 53.97060224186233
Recall: 66.31399035432867
F1 Score: 52.30382225883656


In [32]:
#SVC
model2 = svm.SVC(kernel='linear')
model2 = model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred)*100)
print("Precision:", precision_score(y_test, y_pred, average='macro')*100)
print("Recall:", recall_score(y_test, y_pred, average='macro')*100)
print("F1 Score:", f1_score(y_test, y_pred, average='macro')*100)

Accuracy: 87.47865680136596
Precision: 29.159552267121985
Recall: 33.33333333333333
F1 Score: 31.107063347500507


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
#KNN
model3 = KNeighborsClassifier(n_neighbors=5)
model3.fit(X_train, y_train)
y_pred = model3.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred)*100)
print("Precision:", precision_score(y_test, y_pred, average='macro')*100)
print("Recall:", recall_score(y_test, y_pred, average='macro')*100)
print("F1 Score:", f1_score(y_test, y_pred, average='macro')*100)

Accuracy: 89.2430278884462
Precision: 70.37732549374341
Recall: 72.12918395179182
F1 Score: 70.35834684385016
